# 节点分类

In [1]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

# GNN 节点分类总览

## Loading Cora Dataset

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
7


In [3]:
dataset

Dataset("cora_v2", num_graphs=1, save_path=/Users/qinzijian/.dgl/cora_v2_d697a464)

In [4]:
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

DGL图将节点特征和边特征保存于两个字典样的属性中，即`ndata`和`edata`  
其中，`ndata`中包含5个节点特征
 - `train_mask` Boolean tensor 表示节点是否为训练集
 - `val_mask`
 - `test_mask`
 - `label` 节点的真实类别
 - `feat` 节点特征

In [10]:
g.ndata

{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}

In [6]:
g.edata

{}

In [17]:
g.ndata['feat']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [18]:
g.ndata['train_mask']

tensor([ True,  True,  True,  ..., False, False, False])

# 定义图卷积网络（GCN）

本教程中，建立两层的 GCN，每层通过聚集相邻的信息来计算新的节点表示

In [21]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [22]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [23]:
g.ndata['feat'].shape[1]

1433

DGL 提供了许多流行的相邻聚合模块的实现，可以用一行代码轻松调用它们

# 训练 GCN

In [24]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(100):
        # forward
        logits = model(g, features)
        
        # compute prediction
        pred = logits.argmax(1)
        
        # compute loss
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # compute accuracy
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # update best acc
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if e % 5 == 0:
            print('In epoch {}, loss {:.3f}, val acc {:.3f} (best {:.3f}), test acc {:.3f} (best {:.3f})'\
                 .format(e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [25]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss 1.946, val acc 0.114 (best 0.114), test acc 0.098 (best 0.098)
In epoch 5, loss 1.891, val acc 0.552 (best 0.556), test acc 0.551 (best 0.565)
In epoch 10, loss 1.807, val acc 0.628 (best 0.628), test acc 0.645 (best 0.645)
In epoch 15, loss 1.700, val acc 0.668 (best 0.668), test acc 0.692 (best 0.692)
In epoch 20, loss 1.568, val acc 0.676 (best 0.678), test acc 0.714 (best 0.711)
In epoch 25, loss 1.416, val acc 0.702 (best 0.702), test acc 0.736 (best 0.736)
In epoch 30, loss 1.249, val acc 0.720 (best 0.720), test acc 0.745 (best 0.745)
In epoch 35, loss 1.076, val acc 0.738 (best 0.738), test acc 0.751 (best 0.746)
In epoch 40, loss 0.905, val acc 0.748 (best 0.748), test acc 0.764 (best 0.761)
In epoch 45, loss 0.746, val acc 0.748 (best 0.750), test acc 0.764 (best 0.764)
In epoch 50, loss 0.606, val acc 0.752 (best 0.752), test acc 0.768 (best 0.764)
In epoch 55, loss 0.487, val acc 0.758 (best 0.758), test acc 0.773 (best 0.773)
In epoch 60, loss 0.391, val a

## 分解

In [27]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']

In [29]:
logits = model(g, features)
logits

tensor([[ 0.0029,  0.0030, -0.0056,  ..., -0.0014,  0.0072, -0.0019],
        [-0.0068,  0.0143, -0.0024,  ..., -0.0038, -0.0057, -0.0033],
        [ 0.0008,  0.0058, -0.0058,  ...,  0.0030,  0.0054, -0.0086],
        ...,
        [ 0.0074,  0.0002,  0.0046,  ...,  0.0048,  0.0085, -0.0111],
        [ 0.0048, -0.0012, -0.0050,  ...,  0.0037,  0.0051, -0.0023],
        [ 0.0032,  0.0024, -0.0045,  ..., -0.0007,  0.0029, -0.0031]],
       grad_fn=<AddBackward0>)

In [30]:
logits.shape

torch.Size([2708, 7])

In [31]:
pred = logits.argmax(1)
pred

tensor([5, 1, 1,  ..., 5, 3, 3])

In [32]:
pred.shape

torch.Size([2708])

In [33]:
loss = F.cross_entropy(logits[train_mask], labels[train_mask])
loss

tensor(1.9460, grad_fn=<NllLossBackward0>)

In [34]:
(pred[train_mask] == labels[train_mask])

tensor([False, False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False,  True,  True,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False,  True, False, False,  True, False,  True,  True, 

In [35]:
(pred[train_mask] == labels[train_mask]).float()

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.])